**Часть 1**

Нужно подобрать оптимальные параметры xgboost для набора данных HR.csv. Код для проверки качества представлен в скрипте xgboost_params_checker.py, а пример набора параметров в xgboost_params_example.json. Чекер с вашими параметрами должен отработать за 2 минуты на машинке для проверки. Для сравнения на xgboost_params_example.json чекер работает 20 секунд.


In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
data = pd.read_csv('HR.csv', header = 0, sep = ',')

In [3]:
X = data.drop('left', 1)
y = data['left']
#несбалансированные классы
y.value_counts(normalize=True)

0    0.654777
1    0.345223
Name: left, dtype: float64

In [4]:
# пропущенных значений нет:
#data.info()
# некоторые признаки (last_evaluation, number_project, average_montly_hours) скоррелированы довольно значительно:
#data.corr().head()
# признаки сильно отличаются друг от друга по модулю значений - но масшабирование не сыграло роли в этой задаче:
#data.describe()

In [5]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
%%time
estimator = XGBClassifier(learning_rate=0.1, max_depth=5, n_estimators=1000, min_child_weight=3, seed = 42)
score = cross_val_score(estimator, X, y, scoring = 'accuracy', cv = 3).mean()
print (score, np.mean(score))
# базовые параметры

0.7663200640128025 0.7663200640128025
CPU times: user 11.1 s, sys: 43.4 ms, total: 11.2 s
Wall time: 11.2 s


In [8]:
%%time
wasbest = XGBClassifier(learning_rate=0.005, max_depth=7, min_child_weight=7,
                             n_estimators = 750, seed = 42,
                             objective = 'binary:logistic', scale_pos_weight=0.8,
                             gamma = 0.1, reg_alpha = 0.218,
                             subsample = 1, colsample_bytree = 0.7)
score = cross_val_score(wasbest, X, y, scoring = 'accuracy', cv = 3)
print (score, np.mean(score))
#Более оптимальные параметры

[0.873      0.6508     0.82616523] 0.7833217443488697
CPU times: user 10.4 s, sys: 23.9 ms, total: 10.4 s
Wall time: 10.4 s


In [9]:
#проверка баллов
max(round(2 ** (30 * (0.7833217443488697 - 0.783)), 2), 0.05)

1.01